In [22]:
import pandas as pd
import numpy as np

def customized_pagerank(relations_df, max_iterations=100, alpha=0.85, tolerance=1e-6):
    """
    多加了两个argument：
      alpha: 阻尼因子
      tolerance: 收敛值

    返回：各文档的信任得分
    """
    # 提取所有的文档 ID
    document_ids = set(relations_df['doc_id1']).union(set(relations_df['doc_id2']))
    document_ids = sorted(document_ids)
    doc_index = {doc_id: idx for idx, doc_id in enumerate(document_ids)}
    index_doc = {idx: doc_id for doc_id, idx in doc_index.items()}
    N = len(document_ids)

    # 初始化信任度向量 s_d0
    s = np.full(N, 0.5)
    s0 = s.copy()

    # 添加可信文档
    trusted_docs = []
    for doc_id in trusted_docs:
        if doc_id in doc_index:  # 检查是否在 doc_index 中
            idx = doc_index[doc_id]
            s[idx] = 1.0
            s0[idx] = 1.0

    # 正向和负向的邻接矩阵
    W_plus = np.zeros((N, N))
    W_minus = np.zeros((N, N))

    # 填充实际关系
    for _, row in relations_df.iterrows():
        doc_id1 = row['doc_id1']
        doc_id2 = row['doc_id2']
        normalized_relation = row['normalized_relation']
        i = doc_index[doc_id1]
        j = doc_index[doc_id2]
        if normalized_relation > 0:
            W_plus[i, j] += normalized_relation
        elif normalized_relation < 0:
            W_minus[i, j] -= normalized_relation

    # 添加默认关系
    default_relation = 0.001
    for i in range(N):
        for j in range(N):
            if i != j and (document_ids[i], document_ids[j]) not in relations_df[['doc_id1', 'doc_id2']].values:
                W_plus[i, j] += default_relation

    # 计算每个文档的正向和负向权重总和
    W_plus_sum = W_plus.sum(axis=0)
    W_minus_sum = W_minus.sum(axis=0)

    for iteration in range(max_iterations):
        s_prev = s.copy()
        P_pos = np.zeros(N)
        N_neg = np.zeros(N)

        # 正负向影响
        for d in range(N):
            if W_plus_sum[d] > 0:
                P_pos[d] = np.dot(W_plus[:, d], s_prev) / W_plus_sum[d]
            else:
                P_pos[d] = 0

            if W_minus_sum[d] > 0:
                N_neg[d] = np.dot(W_minus[:, d], s_prev) / W_minus_sum[d]
            else:
                N_neg[d] = 0

        # 净影响 + 映射
        I = P_pos - N_neg
        f_I = (I + 1) / 2
        s = (1 - alpha) * s0 + alpha * f_I

        # 收敛判断
        delta = np.linalg.norm(s - s_prev, ord=1)
        if delta < tolerance:
            print(f'迭代在第 {iteration+1} 次时收敛，变化量 {delta}')
            break

    scores = {index_doc[idx]: s[idx] for idx in range(N)}
    return scores


In [15]:
def claim_relations_to_doc_relations(claim_relations_df):
  claim_relations_df['doc_id1'] = claim_relations_df['id1'].str[:4]
  claim_relations_df['doc_id2'] = claim_relations_df['id2'].str[:4]

  doc_relation_counts = claim_relations_df.groupby(['doc_id1', 'doc_id2'])['relation'].agg(
      support_count = lambda x: (x == 1).sum(),
      oppose_count = lambda x: (x == -1).sum()
  ).reset_index()

  doc_relation_counts['net_relation'] = doc_relation_counts['support_count'] - doc_relation_counts['oppose_count']
  max_net_relation = doc_relation_counts['net_relation'].abs().max()
  doc_relation_counts['normalized_relation'] = doc_relation_counts['net_relation'] / max_net_relation if max_net_relation != 0 else 0

  document_relations_df = doc_relation_counts[['doc_id1', 'doc_id2', 'normalized_relation']]

  return document_relations_df

In [3]:
!rm -rf ClaimRank
!git clone https://github.com/Averyyy/ClaimRank

Cloning into 'ClaimRank'...
remote: Enumerating objects: 226, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 226 (delta 37), reused 57 (delta 18), pack-reused 141 (from 1)
Receiving objects: 100% (226/226), 23.87 MiB | 8.60 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [23]:
import pandas as pd

df = pd.read_csv('ClaimRank/dataset/relations_gemma2_27b_0.009_0-1000.csv')
df = pd.read_csv('ClaimRank/dataset/relations_gemma2_27b_0.009_0-1000.csv', usecols=['id1', 'id2', 'relation'], dtype={'id1': str, 'id2': str})
print(df.columns)
document_relations_df = claim_relations_to_doc_relations(df)
print(document_relations_df.columns)
scores = customized_pagerank(document_relations_df, max_iterations=10)
for doc_id, score in scores.items():
    print(f'文档 {doc_id} 的信任得分为 {score:.4f}')


Index(['id1', 'id2', 'relation'], dtype='object')
Index(['doc_id1', 'doc_id2', 'normalized_relation'], dtype='object')
文档 0000 的信任得分为 0.8695
文档 0001 的信任得分为 0.8695
文档 0002 的信任得分为 0.8695
文档 0003 的信任得分为 0.8695
文档 0004 的信任得分为 0.8695
文档 0005 的信任得分为 0.8695
文档 0006 的信任得分为 0.5000
文档 0007 的信任得分为 0.5785
文档 0008 的信任得分为 0.4096
文档 0009 的信任得分为 0.8695
文档 0010 的信任得分为 0.6583
文档 0012 的信任得分为 0.6371
文档 0013 的信任得分为 0.7648
文档 0015 的信任得分为 0.8019
文档 0017 的信任得分为 0.5935
文档 0018 的信任得分为 0.2875
文档 0019 的信任得分为 0.8695
文档 0020 的信任得分为 0.6570
文档 0021 的信任得分为 0.8321
文档 0022 的信任得分为 0.8695
文档 0026 的信任得分为 0.5000
文档 0027 的信任得分为 0.4686
文档 0028 的信任得分为 0.8695
文档 0030 的信任得分为 0.7743
文档 0032 的信任得分为 0.4904
文档 0035 的信任得分为 0.4215
文档 0041 的信任得分为 0.8695
文档 0044 的信任得分为 0.8695
文档 0047 的信任得分为 0.8695
文档 0050 的信任得分为 0.8166
文档 0052 的信任得分为 0.8695
文档 0053 的信任得分为 0.8695
文档 0054 的信任得分为 0.1698
文档 0056 的信任得分为 0.8695
文档 0057 的信任得分为 0.8346
文档 0058 的信任得分为 0.4751
文档 0059 的信任得分为 0.8302
文档 0060 的信任得分为 0.8433
文档 0061 的信任得分为 0.7814
文档 0063 的信任得分为 0.6570
文

In [24]:
for doc_id, score in scores.items():
    if doc_id == '0555':
        print(f'文档 {doc_id} 的信任得分为 {score:.4f}')

文档 0555 的信任得分为 0.8551
